In [1]:
import numpy as np
import seaborn as sns
from time import time

import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.regularizers import L1L2
import keras.backend as K

import os
import gc
from pathlib import Path

tf.config.experimental.set_visible_devices([], 'GPU')

Using TensorFlow backend.


In [2]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [3]:
class Game:
    def __init__(self, state, FIRST):
        self.state = state
        self.empty = self.make_empty(state)
        self.first_player = FIRST
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        
        return emp
    
    def is_lose(self, a):
        opp = 2 - (a-1)
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == opp:
                return 1
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == opp:
                return 1
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == opp:
            return 1
        return 0
    
    def is_win(self, a):       
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
                return 1
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
                return 1
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
            return 1
        if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
            return 1
        return 0
    
    def is_draw(self, a):
        if self.is_win(a):
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_win(1) or self.is_win(2) or self.is_draw(1):
            return 1
        else:
            return 0
        
        
    def update(self, target):
        state = self.state.copy()
        x, y = target//3, target%3
        a = self.next_opp()
        state[x][y] = a
        return Game(state, self.first_player)
    
    
    def next_opp(self):
        a = b = 0
        for i in range(len(self.state)):
            for j in range(len(self.state)):
                if self.state[i][j] == self.first_player:
                    a += 1
                elif self.state[i][j] != 0:
                    b += 1
                    
        if a == b:
            return self.first_player
        else:
            return 2 + min(0, 1-self.first_player)

In [4]:
class Random:
    def __init__(self, status):
        self.status = status

    def action(self, game):
        return np.random.choice(game.empty)

In [5]:
status = 1

def playout(game):
    if game.is_lose(status):
        return -1

    if game.is_draw(status):
        return 0


    if game.is_win(status):
        return 1

    return playout(game.update(np.random.choice(game.empty)))


def action(game):
    values = [0] * len(game.empty)

    for i, a in enumerate(game.empty):
        for _ in range(100):
            g = game.update(a)
            values[i] += playout(g)

    return values

In [6]:
DN_FILTERS = 64  # 컨볼루션 레이어 커널 수(오리지널 256）
DN_RESIDUAL_NUM = 8  # 레지듀얼 블록 수(오리지널 19)
DN_INPUT_SHAPE = (1, 3, 3)  # 입력 셰이프
DN_OUTPUT_SIZE = 9  # 행동 수(배치 수(3*3))
    
def residual_block():
    def f(x):
        sc = x
        x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, sc])
        x = layers.Activation('relu')(x)
        return x

    return f
    
def dual_network():
#     # 모델 생성이 완료된 경우 처리하지 않음
#     if os.path.exists('./model/best.h5'):
#         return

    # 입력 레이어
    input = layers.Input(shape=DN_INPUT_SHAPE)

    # 컨볼루션 레이어
    x = layers.Conv2D(DN_FILTERS, 3, padding='same', use_bias=False,
              kernel_initializer='he_normal', kernel_regularizer=L1L2(l2=0.0005))(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    # 레지듀얼 블록 x 16
    for i in range(DN_RESIDUAL_NUM):
        x = residual_block()(x)

    # 풀링 레이어
    x = layers.GlobalAveragePooling2D()(x)

    # policy 출력
    p = layers.Dense(DN_OUTPUT_SIZE, kernel_regularizer=L1L2(l2=0.0005),
              activation='tanh', name='pi')(x)

    # value 출력
#     v = layers.Dense(1, kernel_regularizer=L1L2(l2=0.0005))(x)
#     v = layers.Activation('tanh', name='v')(v)

    # 모델 생성
    model = models.Model(inputs=input, outputs=p)

    model.compile(optimizer = 'adam',
                 loss = 'mse')

    return model

class DQN:
    def __init__(self, status):
        K.clear_session()
#         if os.path.exists('./best.h5'):
#             print('kk')
#             path = sorted(Path('./').glob('*.h5'))[-1]
#             print(path)
#             self.model = models.load_model(str(path))
#         else:
#             self.model = dual_network()
            
        self.model = dual_network()
        self.status = status
        
    def action(self, game):
        status = game.next_opp()
        target = np.reshape(game.state, (1, 1, 3, 3)).astype('float')
        target = np.where(target==status, 1., np.where(target==0, 0, -1.))
        res = self.model.predict(target)[0]
        a = np.argmax(res)
        while a not in game.empty:
            res[a] = -float('inf')
            a = np.argmax(res)
        return a
    
    
    def value(self, game):
        values = [0] * 9
        n_steps = 20

        for i in range(9):
            if i in game.empty:
                for _ in range(n_steps):
                    g = game.update(i)
                    values[i] += playout(g)
            values[i] /= n_steps
            
        return values

    def train(self):
        for i in [1, 2]:
            game = Game(init_state, i)
            X = []
            y = []
            while 1:
                status = game.next_opp()
                state = np.reshape(game.state, (1, 1, 3, 3)).astype('float')
                state = np.where(state==status, 1., np.where(state==0, 0, -1.))

                X.append(state)
                y.append(self.value(game))

                a = self.action(game)
                game = game.update(a)

                if game.is_done():
                    break

            X = np.reshape(X, (len(X), 1, 3, 3))
            y = np.reshape(y, (len(y), 9))   
            self.model.fit(X, y, epochs=1, verbose=0)
                
    
    def opp(self, status):
        return 2 + min(0, 1-status)


In [7]:
dd = DQN(1)

In [8]:
dd.train()

In [9]:
for _ in range(500):
    print(_)
    dd.train()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
dd.model.predict(a)

In [ ]:
FIRST = 1
d = DQN(1)

In [ ]:
a = init_state.copy()
a = np.reshape(a, (1, 1, 3, 3)).astype('float')
a = np.where(a==1, 1., np.where(a==0, 0, -1.))

dd.model.predict(a)

In [ ]:
gc.collect()

In [ ]:
# def train(game):
#     global X, y

#     while 1:
#         X.append(np.reshape(game.state, (1, 3, 3)))
#         a1 = action(game)
#         game = game.update(a1)
#         temp_y = [0] * 9
#         temp_y[a1] = 1
#         y.append(temp_y)
    
#         if game.is_win(1):
#             break 
#         elif game.is_draw(1):
#             break 

#         a2 = np.random.choice(game.empty)
#         game = game.update(a2)
#         if game.is_win(2):
#             y *= -1
#             break 
#         elif game.is_draw(2):
#             break
        
#     X = np.reshape(X, (len(X), 1, 3, 3))
#     if len(y) == 0:
#         print('pass')
#         return
#     y = np.reshape(y, (len(X), 9))   

#     k.model.fit(X, y, epochs=1, verbose=0)
# #     return model

In [ ]:
# %%time
# # train n_steps
# FIRST = 1
# k = DQN(1)
# # model = k.model
# g = Game(init_state)
# n_steps = 500
# for _ in range(n_steps):
#     X = []
#     y = []
#     train(g)
#     gc.collect()
#     del X, y

In [10]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        if a1 not in game.empty:
            print('fail')
            return
        game = game.update(a1)
        if game.is_win(m1.status):
            score[m1.status-1] += 1
            print(game.state)
            return 
        elif game.is_draw(m1.status):
            score[2] += 1
            print(game.state)
            return 

        a2 = m2.action(game)
        if a2 not in game.empty:
            print('fail')
            return
        game = game.update(a2)
        if game.is_win(m2.status):
            score[m2.status-1] += 1
            print(game.state)
            return 
        elif game.is_draw(m2.status):
            score[2] += 1
            print(game.state)
            return 
        

In [11]:
dd.model.predict(np.reshape(init_state, (1, 1, 3, 3))).astype('f')[0]

array([-0.15265132,  0.40122542, -0.25976166,  0.14091292,  0.05102095,
        0.2256625 ,  0.28851327,  0.318053  , -0.08055871], dtype=float32)

In [15]:
%%time

m1 = Random(2)
score = [0, 0, 0]

game = Game(init_state, 1)
for _ in range(1000):
    print(_)
    play(game, m1, dd)
    
gc.collect()

game = Game(init_state, 2)
score1 = score.copy()
for _ in range(1000):
    print(_)
    play(game, dd, m1)

0
[[1 2 1]
 [1 2 2]
 [1 2 1]]
1
[[1 2 1]
 [1 2 1]
 [2 1 2]]
2
[[1 2 1]
 [2 1 2]
 [2 1 1]]
3
[[1 2 1]
 [1 2 2]
 [2 1 1]]
4
[[1 2 1]
 [1 2 2]
 [2 1 1]]
5
[[1 2 0]
 [1 2 1]
 [1 2 0]]
6
[[1 2 1]
 [2 1 1]
 [1 2 2]]
7
[[1 2 1]
 [1 1 2]
 [2 2 1]]
8
[[1 2 0]
 [2 2 2]
 [1 1 1]]
9
[[1 2 0]
 [1 1 2]
 [1 2 2]]
10
[[2 2 1]
 [1 2 1]
 [2 1 1]]
11
[[1 2 1]
 [1 1 2]
 [2 1 2]]
12
[[1 2 1]
 [1 1 2]
 [2 2 1]]
13
[[1 2 1]
 [1 2 2]
 [2 1 1]]
14
[[1 2 0]
 [1 1 2]
 [1 2 2]]
15
[[1 2 0]
 [2 1 2]
 [2 1 1]]
16
[[1 2 1]
 [1 1 2]
 [2 2 1]]
17
[[1 2 1]
 [1 2 2]
 [2 1 1]]
18
[[1 2 1]
 [0 2 1]
 [1 2 0]]
19
[[1 1 1]
 [2 0 1]
 [2 2 2]]
20
[[1 2 1]
 [1 1 2]
 [2 1 2]]
21
[[1 2 1]
 [1 2 1]
 [2 1 2]]
22
[[1 2 1]
 [1 2 2]
 [1 2 1]]
23
[[1 1 1]
 [2 1 1]
 [2 2 2]]
24
[[1 2 0]
 [1 2 2]
 [1 1 2]]
25
[[1 2 1]
 [1 2 2]
 [2 1 1]]
26
[[1 2 1]
 [1 1 1]
 [2 2 2]]
27
[[1 2 1]
 [1 1 2]
 [2 2 1]]
28
[[1 2 1]
 [2 1 2]
 [1 2 0]]
29
[[2 1 1]
 [1 1 2]
 [2 2 1]]
30
[[1 2 1]
 [2 2 2]
 [1 1 1]]
31
[[1 2 1]
 [1 2 2]
 [2 1 1]]
32
[[0 2 2]
 [1 2 

[[1 2 1]
 [1 1 2]
 [2 2 1]]
263
[[1 2 1]
 [1 2 2]
 [2 1 1]]
264
[[2 2 1]
 [1 1 2]
 [1 2 1]]
265
[[1 2 1]
 [0 2 1]
 [2 2 1]]
266
[[1 2 1]
 [1 2 1]
 [2 2 1]]
267
[[1 2 0]
 [0 1 2]
 [2 0 1]]
268
[[1 2 1]
 [1 2 2]
 [2 1 1]]
269
[[1 1 1]
 [0 0 2]
 [2 2 0]]
270
[[1 2 1]
 [1 1 2]
 [2 2 1]]
271
[[2 1 1]
 [2 0 1]
 [2 2 1]]
272
[[1 2 1]
 [2 1 1]
 [1 2 2]]
273
[[1 2 1]
 [1 1 2]
 [2 1 2]]
274
[[0 2 0]
 [1 1 1]
 [2 2 0]]
275
[[1 2 1]
 [1 1 2]
 [2 2 1]]
276
[[1 2 1]
 [1 1 2]
 [2 1 2]]
277
[[1 2 0]
 [2 1 2]
 [2 1 1]]
278
[[1 2 1]
 [1 2 2]
 [1 2 1]]
279
[[1 1 1]
 [1 2 2]
 [2 2 1]]
280
[[1 1 1]
 [1 2 2]
 [2 2 1]]
281
[[1 2 1]
 [0 2 1]
 [2 2 1]]
282
[[0 2 1]
 [0 1 2]
 [1 2 0]]
283
[[1 2 1]
 [2 2 1]
 [2 1 1]]
284
[[2 2 0]
 [1 1 1]
 [2 1 2]]
285
[[0 2 1]
 [2 1 1]
 [2 2 1]]
286
[[1 1 1]
 [1 2 2]
 [2 2 1]]
287
[[1 2 0]
 [2 1 2]
 [2 1 1]]
288
[[2 2 0]
 [1 1 1]
 [2 2 1]]
289
[[1 2 1]
 [2 2 1]
 [2 1 1]]
290
[[2 2 1]
 [1 1 2]
 [1 2 0]]
291
[[1 2 0]
 [1 2 2]
 [1 0 0]]
292
[[1 2 1]
 [1 2 1]
 [2 1 2]]
293
[[2 2 1]

[[2 2 1]
 [1 1 2]
 [1 2 0]]
532
[[1 2 1]
 [1 2 1]
 [2 1 2]]
533
[[1 2 1]
 [1 1 2]
 [2 2 1]]
534
[[1 2 1]
 [2 2 2]
 [1 1 1]]
535
[[1 2 1]
 [1 2 1]
 [2 1 2]]
536
[[1 2 1]
 [1 1 2]
 [2 2 1]]
537
[[2 2 1]
 [0 1 2]
 [1 2 1]]
538
[[2 2 1]
 [0 1 2]
 [1 2 1]]
539
[[0 2 1]
 [1 2 1]
 [2 2 1]]
540
[[1 2 1]
 [1 2 2]
 [2 1 1]]
541
[[0 2 1]
 [2 2 1]
 [2 1 1]]
542
[[2 1 1]
 [1 1 2]
 [2 2 1]]
543
[[0 2 1]
 [1 2 1]
 [2 2 1]]
544
[[1 2 1]
 [1 1 2]
 [2 1 2]]
545
[[1 2 1]
 [1 2 2]
 [2 1 1]]
546
[[0 2 1]
 [0 1 2]
 [1 2 0]]
547
[[1 2 1]
 [1 2 2]
 [2 1 1]]
548
[[2 1 1]
 [2 1 2]
 [1 2 0]]
549
[[1 2 1]
 [2 2 1]
 [2 1 1]]
550
[[1 2 1]
 [1 2 2]
 [1 2 1]]
551
[[1 1 1]
 [2 1 2]
 [1 2 2]]
552
[[1 2 1]
 [1 2 1]
 [2 1 2]]
553
[[2 2 1]
 [1 1 1]
 [2 2 0]]
554
[[1 2 1]
 [1 2 2]
 [1 2 1]]
555
[[1 2 1]
 [2 2 1]
 [1 2 1]]
556
[[0 2 0]
 [0 2 2]
 [1 1 1]]
557
[[1 2 0]
 [1 1 2]
 [2 2 1]]
558
[[1 1 1]
 [0 0 2]
 [2 2 0]]
559
[[1 2 0]
 [2 1 2]
 [2 1 1]]
560
[[1 2 1]
 [1 1 2]
 [2 2 1]]
561
[[1 2 1]
 [1 2 1]
 [1 2 2]]
562
[[2 1 1]

[[1 2 1]
 [1 1 2]
 [2 2 1]]
801
[[1 2 1]
 [1 1 2]
 [2 2 1]]
802
[[1 2 1]
 [2 1 2]
 [2 1 1]]
803
[[0 2 1]
 [2 1 1]
 [1 2 2]]
804
[[1 2 1]
 [1 1 2]
 [2 1 2]]
805
[[1 2 0]
 [1 1 2]
 [2 2 1]]
806
[[1 2 1]
 [1 2 1]
 [2 2 1]]
807
[[1 2 1]
 [1 2 2]
 [2 1 1]]
808
[[1 2 0]
 [0 1 2]
 [2 0 1]]
809
[[1 2 0]
 [1 2 2]
 [1 1 2]]
810
[[1 2 1]
 [0 1 2]
 [2 2 1]]
811
[[0 2 0]
 [1 1 1]
 [2 2 0]]
812
[[1 2 1]
 [1 1 2]
 [2 1 2]]
813
[[1 2 0]
 [2 1 1]
 [2 2 1]]
814
[[1 2 0]
 [1 2 2]
 [1 1 2]]
815
[[1 2 1]
 [1 1 2]
 [2 2 1]]
816
[[1 2 1]
 [1 2 2]
 [2 1 1]]
817
[[1 1 1]
 [1 2 2]
 [2 2 0]]
818
[[1 2 0]
 [1 1 2]
 [1 2 2]]
819
[[0 2 1]
 [0 1 2]
 [1 2 0]]
820
[[1 2 1]
 [1 2 2]
 [2 1 1]]
821
[[1 2 1]
 [1 1 2]
 [2 2 1]]
822
[[2 2 1]
 [2 0 1]
 [1 2 1]]
823
[[1 2 0]
 [1 2 2]
 [1 0 0]]
824
[[1 2 1]
 [1 2 2]
 [2 1 1]]
825
[[1 2 1]
 [1 1 2]
 [2 2 1]]
826
[[2 2 1]
 [2 0 1]
 [1 2 1]]
827
[[1 2 1]
 [1 2 2]
 [2 1 1]]
828
[[1 1 1]
 [2 1 2]
 [2 2 1]]
829
[[1 2 1]
 [1 2 1]
 [2 1 2]]
830
[[1 2 1]
 [2 2 2]
 [1 1 1]]
831
[[1 2 0]

 [2 2 1]]
68
[[1 2 1]
 [0 2 1]
 [2 2 1]]
69
[[1 2 1]
 [2 2 2]
 [2 1 1]]
70
[[1 2 0]
 [0 1 2]
 [2 2 1]]
71
[[2 2 1]
 [2 1 1]
 [2 2 1]]
72
[[1 2 0]
 [1 1 1]
 [2 2 2]]
73
[[1 2 1]
 [0 2 2]
 [1 2 1]]
74
[[1 2 1]
 [1 1 2]
 [2 2 2]]
75
[[2 2 1]
 [1 1 2]
 [1 2 2]]
76
[[1 2 1]
 [2 2 1]
 [2 1 2]]
77
[[1 2 1]
 [2 1 2]
 [1 2 2]]
78
[[2 2 1]
 [2 1 1]
 [2 2 1]]
79
[[1 2 0]
 [1 2 1]
 [2 2 1]]
80
[[1 2 2]
 [2 1 1]
 [2 2 1]]
81
[[0 2 1]
 [2 0 1]
 [2 2 1]]
82
[[1 2 1]
 [2 1 2]
 [2 1 2]]
83
[[0 2 1]
 [1 2 1]
 [2 2 1]]
84
[[1 2 1]
 [0 2 2]
 [1 2 1]]
85
[[2 2 1]
 [2 1 1]
 [2 2 1]]
86
[[2 2 2]
 [1 1 2]
 [1 2 1]]
87
[[0 2 0]
 [1 1 1]
 [2 2 2]]
88
[[2 2 1]
 [2 1 1]
 [2 2 1]]
89
[[1 2 2]
 [1 1 2]
 [2 2 1]]
90
[[0 2 1]
 [1 2 1]
 [2 2 1]]
91
[[1 2 1]
 [0 2 2]
 [1 2 1]]
92
[[1 2 1]
 [1 2 2]
 [2 2 1]]
93
[[0 2 1]
 [1 2 2]
 [1 2 1]]
94
[[1 2 0]
 [1 2 2]
 [1 2 1]]
95
[[1 2 0]
 [1 2 2]
 [1 2 0]]
96
[[2 2 1]
 [1 2 1]
 [2 1 2]]
97
[[1 2 2]
 [2 1 1]
 [2 2 1]]
98
[[1 2 1]
 [1 2 1]
 [2 2 2]]
99
[[0 2 1]
 [2 0 1]
 [2 2 1]

 [2 2 1]]
336
[[2 2 1]
 [2 1 2]
 [1 2 1]]
337
[[1 2 1]
 [1 2 2]
 [2 2 1]]
338
[[1 2 1]
 [2 1 2]
 [2 2 1]]
339
[[1 2 0]
 [1 1 1]
 [2 2 2]]
340
[[1 2 1]
 [2 2 1]
 [2 1 2]]
341
[[2 2 1]
 [2 2 1]
 [2 1 1]]
342
[[1 2 0]
 [0 1 2]
 [2 2 1]]
343
[[2 2 1]
 [1 2 1]
 [2 1 2]]
344
[[1 2 1]
 [1 2 2]
 [1 2 0]]
345
[[1 2 1]
 [1 1 2]
 [2 2 2]]
346
[[2 2 1]
 [2 2 1]
 [2 1 1]]
347
[[2 2 2]
 [1 1 2]
 [1 2 1]]
348
[[1 2 2]
 [1 1 2]
 [2 2 1]]
349
[[1 2 1]
 [1 2 2]
 [1 2 0]]
350
[[1 2 1]
 [2 1 2]
 [2 2 1]]
351
[[2 2 1]
 [1 1 2]
 [2 2 1]]
352
[[2 2 2]
 [1 1 2]
 [1 2 1]]
353
[[1 2 2]
 [1 1 2]
 [2 1 2]]
354
[[1 2 1]
 [2 1 2]
 [2 2 1]]
355
[[1 2 1]
 [1 2 2]
 [2 1 2]]
356
[[1 2 1]
 [2 1 2]
 [2 2 1]]
357
[[0 2 1]
 [1 2 2]
 [1 2 1]]
358
[[1 2 1]
 [1 2 1]
 [2 2 2]]
359
[[1 2 1]
 [1 2 1]
 [2 2 2]]
360
[[1 2 1]
 [2 2 2]
 [2 1 1]]
361
[[1 2 2]
 [1 1 2]
 [2 1 2]]
362
[[2 2 1]
 [1 1 2]
 [2 1 2]]
363
[[1 2 1]
 [2 1 2]
 [2 2 1]]
364
[[1 2 1]
 [2 2 2]
 [2 1 1]]
365
[[1 2 2]
 [1 2 1]
 [2 1 2]]
366
[[1 2 0]
 [1 2 2]
 [1 2 0]

 [2 1 2]]
593
[[1 2 1]
 [2 1 2]
 [2 2 1]]
594
[[1 2 1]
 [1 1 2]
 [2 2 2]]
595
[[1 2 2]
 [1 1 2]
 [2 2 1]]
596
[[1 2 1]
 [1 1 2]
 [2 2 2]]
597
[[0 2 1]
 [2 1 2]
 [1 2 0]]
598
[[1 2 1]
 [2 1 2]
 [2 1 2]]
599
[[2 2 1]
 [1 2 2]
 [2 1 1]]
600
[[1 2 2]
 [1 1 2]
 [2 2 1]]
601
[[1 2 0]
 [1 2 2]
 [1 2 1]]
602
[[1 2 1]
 [1 2 2]
 [0 2 1]]
603
[[1 2 1]
 [2 1 2]
 [2 2 1]]
604
[[1 2 1]
 [1 2 2]
 [2 2 1]]
605
[[1 2 1]
 [1 1 2]
 [2 2 2]]
606
[[1 2 1]
 [2 1 2]
 [2 2 1]]
607
[[1 2 1]
 [2 2 1]
 [2 1 2]]
608
[[2 2 2]
 [2 1 1]
 [1 2 1]]
609
[[1 2 2]
 [2 1 1]
 [2 2 1]]
610
[[1 2 0]
 [1 2 2]
 [1 2 0]]
611
[[1 2 0]
 [0 1 2]
 [2 2 1]]
612
[[1 2 0]
 [1 2 2]
 [1 2 1]]
613
[[2 2 1]
 [2 1 1]
 [2 2 1]]
614
[[1 2 1]
 [1 2 2]
 [1 2 0]]
615
[[2 2 1]
 [1 1 2]
 [2 1 2]]
616
[[2 2 1]
 [1 1 2]
 [2 2 1]]
617
[[1 2 2]
 [1 1 2]
 [2 2 1]]
618
[[0 2 1]
 [2 1 2]
 [1 2 0]]
619
[[1 2 0]
 [1 2 2]
 [1 2 1]]
620
[[2 2 1]
 [2 1 2]
 [1 2 1]]
621
[[2 2 1]
 [1 1 2]
 [2 2 1]]
622
[[1 2 1]
 [1 2 2]
 [2 1 2]]
623
[[0 2 1]
 [1 2 2]
 [1 2 1]

[[2 2 1]
 [2 1 1]
 [2 2 1]]
852
[[1 2 1]
 [1 2 2]
 [2 2 1]]
853
[[2 2 1]
 [1 1 2]
 [2 2 1]]
854
[[1 2 1]
 [1 2 2]
 [2 2 1]]
855
[[2 2 2]
 [1 1 2]
 [1 2 1]]
856
[[1 2 1]
 [1 2 2]
 [2 1 2]]
857
[[1 2 1]
 [1 1 2]
 [2 2 2]]
858
[[1 2 2]
 [1 1 2]
 [2 2 1]]
859
[[1 2 0]
 [1 2 1]
 [2 2 1]]
860
[[2 2 1]
 [2 1 1]
 [2 2 1]]
861
[[1 2 1]
 [1 2 2]
 [2 1 2]]
862
[[1 2 0]
 [1 2 2]
 [1 2 1]]
863
[[0 2 1]
 [1 2 2]
 [1 2 1]]
864
[[2 2 1]
 [0 1 2]
 [1 2 0]]
865
[[1 2 2]
 [2 1 2]
 [1 2 1]]
866
[[1 2 1]
 [1 2 2]
 [1 2 0]]
867
[[1 2 1]
 [1 2 2]
 [0 2 1]]
868
[[1 2 1]
 [2 1 2]
 [2 1 2]]
869
[[1 2 0]
 [0 1 2]
 [2 2 1]]
870
[[1 2 2]
 [1 2 1]
 [2 1 2]]
871
[[1 2 1]
 [0 2 2]
 [1 2 1]]
872
[[0 2 1]
 [2 1 2]
 [1 2 0]]
873
[[0 2 1]
 [2 0 1]
 [2 2 1]]
874
[[1 2 1]
 [2 1 2]
 [2 1 2]]
875
[[2 2 1]
 [1 2 1]
 [2 1 2]]
876
[[1 2 1]
 [1 2 2]
 [2 1 2]]
877
[[1 2 1]
 [0 2 1]
 [2 2 1]]
878
[[1 2 1]
 [2 1 1]
 [2 2 2]]
879
[[0 2 1]
 [2 0 1]
 [2 2 1]]
880
[[1 2 1]
 [2 2 1]
 [1 2 2]]
881
[[2 2 1]
 [2 1 1]
 [1 2 2]]
882
[[2 2 1]

In [16]:
score

[738, 399, 863]

In [17]:
score1

[411, 149, 440]

In [ ]:
sns.barplot(x = [1, 2], y = score[:2])

In [ ]:
# tanh + mse
# v1: 732vs601 // 350vs323
# v2: 738vs399 // 411vs149